## 11. HNSW参数调优难？掌握SQ8量化压缩技术实现速度与准确率平衡

在使用 Faiss 的 HNSW（Hierarchical Navigable Small World）索引时，虽然其搜索速度和准确率都很高，但内存占用大 是其主要缺点之一。

为了解决这个问题，Faiss 提供了 SQ8（Scalar Quantization 8 bits）量化压缩技术 ，可以在几乎不损失精度的前提下显著减少内存占用。

下面我们将通过一个完整的示例，展示：

1. 原始 HNSW 索引的构建与搜索；
2. 使用 SQ8 量化后的 HNSW 构建与搜索；
3. 比较两者在 内存占用、搜索速度、召回率 上的表现。

### 1 安装 Faiss

确保你安装的是支持 GPU 和 SQ8 的版本

In [1]:
%pip install faiss-gpu  # 或 faiss-cpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
Note: you may need to restart the kernel to use updated packages.


## 2. 示例代码：HNSW vs HNSW + SQ8 对比

In [2]:
import numpy as np
import faiss
import time

# Set random seed for reproducibility
np.random.seed(42)

# Construct the dataset
d = 128                             # Vector dimension
nb = 100000                         # Database size
nq = 1000                           # Number of queries
xb = np.random.random((nb, d)).astype('float32')
xq = np.random.random((nq, d)).astype('float32')

# Build the original HNSW index
index_hnsw = faiss.IndexHNSWFlat(d, 32)
index_hnsw.hnsw.efConstruction = 200
index_hnsw.verbose = False

print("Start training original HNSW...")
start_time = time.time()
index_hnsw.add(xb)
end_time = time.time()
print(f"Original HNSW training time: {end_time - start_time:.2f}s")

# Query test
k = 10
D, I = index_hnsw.search(xq, k)
print("Original HNSW top-10 query results (first 5 queries):")
print(I[:5])

# Estimate memory usage
print(
    f"Original HNSW memory usage: "
    f"{index_hnsw.ntotal * index_hnsw.d * 4 / (1024 ** 2):.2f} MB"
)

# Build HNSW + SQ8 index
res = faiss.StandardGpuResources() if 'gpu' in faiss.__version__ else None

# Use HNSW as the quantizer
quantizer = faiss.IndexHNSWFlat(d, 32)
quantizer.hnsw.efConstruction = 200

# Create an index with SQ8 (IVFPQ with 8-bit subquantizers)
index_sq8 = faiss.IndexIVFPQ(
    quantizer,
    d,
    nb // 100,   # Number of inverted lists
    4,           # Number of PQ sub-vectors
    8            # Bits per sub-vector (SQ8)
)
index_sq8.nprobe = 10
index_sq8.verbose = False

# Train the index
print("Start training HNSW+SQ8...")
start_time = time.time()
index_sq8.train(xb)
index_sq8.add(xb)
end_time = time.time()
print(f"HNSW+SQ8 training time: {end_time - start_time:.2f}s")

# Query test
D2, I2 = index_sq8.search(xq, k)
print("HNSW+SQ8 top-10 query results (first 5 queries):")
print(I2[:5])

# Estimate memory usage
# SQ8 uses approximately 1 byte per dimension
print(
    f"HNSW+SQ8 memory usage: "
    f"{index_sq8.ntotal * index_sq8.d * 1 / (1024 ** 2):.2f} MB"
)

# Recall calculation (simple Top-1 comparison)
recall_top1 = np.mean(I[:, 0] == I2[:, 0])
print(f"Top-1 Recall between HNSW and HNSW+SQ8: {recall_top1 * 100:.2f}%")


Start training original HNSW...
Original HNSW training time: 18.46s
Original HNSW top-10 query results (first 5 queries):
[[19523 72727 34414 48303 86082 13870 35989 31498 42753 24473]
 [69743 94079 23747 39882 87413 53658 68352 68848 29454 72974]
 [76117 68518 44977 74433 45084  2909 52787 96987 47724 81358]
 [76328 12625 29097 55150 68382 62868  8947 57611 25164  2698]
 [51507 21281 41110 32283 61565 23634 93393 58215 67148 52124]]
Original HNSW memory usage: 48.83 MB
Start training HNSW+SQ8...
HNSW+SQ8 training time: 3.13s
HNSW+SQ8 top-10 query results (first 5 queries):
[[42687 40153 25478 76836 88638 12691 64882 91012 63804 14203]
 [22830 82772 11575 88026 58855 96105 37560 12570 76578  1969]
 [59046 27092 34814  5230 17828 67396 44235 29962 57022 95544]
 [55411 63904 28105 18244 67250 96963 50741 63394 50041 34580]
 [ 4883 51626 56327 10468 30428 35772 20197 20318 64491 50139]]
HNSW+SQ8 memory usage: 12.21 MB
Top-1 Recall between HNSW and HNSW+SQ8: 0.90%


## 3 HNSW 参数详解

| 参数名            | 默认值     | 含义说明                                     | 调整建议 |
|------------------|------------|----------------------------------------------|----------|
| `M`              | 32         | 每个节点连接的最大邻居数，控制图的复杂度     | 增大 M 提高召回率，但增加内存和构建时间 |
| `efConstruction` | 200        | 构建索引时的搜索范围，影响图质量             | 更高质量 = 更大的 efConstruction，适合数据集复杂或对精度要求高的场景 |
| `efSearch`       | 10         | 搜索时的探索范围，直接影响准确率与速度     | 增大 efSearch 可提升准确率，但会降低查询速度 |
| `level_mult`     | 1 / log(M) | 控制层级分布密度                             | 一般不调整，除非有特殊结构需求 |

适用场景：
- M：图的连接密度
场景：需要高召回率 → 增大 M；
场景：内存敏感、快速部署 → 减小 M；

- efConstruction：构建阶段的质量控制
场景：离线训练、追求精确 → 增大该值；

- efSearch：查询阶段的探索深度
场景：实时性要求高 → 适当减小以加快响应；

- level_mult：层级分布系数
场景：通常默认即可，仅在自定义图结构时调整。

## 4 SQ8 vs SQ16 

SQ8（Scalar Quantization 8-bit）
- 优点：压缩率高，内存最小化；
- 缺点：精度略低；
- 场景：适用于移动端、嵌入式设备、轻量级推荐系统。

SQ16（Scalar Quantization 16-bit）
- 优点：保留更高精度；
- 缺点：内存占用翻倍；
- 场景：适用于云服务、图像检索、视频匹配等对精度要求较高的任务。


## 总结
| 需求类型       | 推荐方案                          | 理由说明                                                                 |
|----------------|-----------------------------------|--------------------------------------------------------------------------|
| **内存敏感**   | HNSW + SQ8                        | 每个向量仅占 1 字节，适合嵌入式/移动端或内存受限环境                       |
| **精度优先**   | HNSW + SQ16 / PQ / OPQ            | 提供更高精度，适合图像检索、视频匹配等对召回率要求高的任务                 |
| **查询速度快** | HNSW + SQ8 + efSearch=20~40       | 在保持合理准确率的同时加快响应速度，适合实时系统                           |
| **大规模部署** | HNSW + SQ8 + GPU 加速             | 支持海量数据快速训练与查询，适用于线上服务                                 |
| **图像特征匹配** | HNSW + SQ8 或 SQ16                | 对于 128D 等常见维度，SQ8 已足够；若需更高精度可用 SQ16                     |
| **视频检索系统** | HNSW + SQ16                       | 视频特征通常更复杂，需要更高精度以提升匹配准确性                           |
| **推荐系统应用** | HNSW + SQ8                        | 在保证性能的前提下节省内存，适合用户/物品 Embedding 的高效匹配              |
| **压缩与精度平衡** | HNSW + PQ / OPQ                  | 使用乘积量化技术，在压缩模型大小的同时保持较高搜索准确率                   |